In [23]:
import random
import tensorflow as tf
import torch
import os
import pandas as pd
# Regular Expression for text cleaning
import re
import numpy as np
from eval_util import *
# to track the progress - progress bar
from tqdm.notebook import tqdm
from tensorflow.keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import contractions

In [29]:

def clean_text(text, mapping):
    replace_white_space = ["\n"]
    for s in replace_white_space:
        text = text.replace(s, " ")
    replace_punctuation = ["’", "‘", "´", "`", "\'", r"\'"]
    for s in replace_punctuation:
        text = text.replace(s, "'")
    
    # Random note: removing the URL's slightly degraded performance, it's possible the model learned that certain URLs were positive/negative
    # And was able to extrapolate that to retweets. Could also explain why re-training the Embeddings improves performance.
    # remove twitter url's
    text = re.sub(r"http[s]?://t.co/[A-Za-z0-9]*","TWITTERURL",text)
    mapped_string = []
    for t in text.split(" "):

        mapped_string.append(contractions.fix(t))  
    return ' '.join(mapped_string)

#training constants
MAX_SEQ_LEN = 36 #this is based on a quick analysis of the len of sequences train['text'].apply(lambda x : len(x.split(' '))).quantile(0.95)
DEFAULT_BATCH_SIZE = 16



# max number of words one sentence can have
MAX_LEN = 36

# width of of 1D embedding vector

##load tensorflow model
tokenizer = Tokenizer(lower=False)
savedModel=load_model('model_1c.h5')

In [30]:


sentence=input("Please enter the sentence:! ")
# sentence = "I really like getting bad marks in college!"
sentence=clean_text(sentence, contractions.fix)
# sentence = preprocessing_text(sentence)
print(sentence)
sentence = tokenizer.texts_to_sequences([sentence])
sentence = pad_sequences(sentence, maxlen = MAX_LEN)
print()
print()
with tf.device('/cpu:0'):
    prediction = savedModel.predict(sentence)
print("The probability of sentence being sarcastic is %.3f percent"%(prediction[0][0]*100))
print()
if prediction[0][0] < 0.3:
    print("You need to work on your sarcasm!")

else:    
    print("you need to spice up your life a bit but nice try!")

Please enter the sentence:! yeah right
yeah right


The probability of sentence being sarcastic is 60.270 percent

you need to spice up your life a bit but nice try!
